<a href="https://colab.research.google.com/github/karaage0703/code-cooker/blob/main/code_cooker_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Cooker
Simple data analysis notebook on google colab like Advanced Data Analysis(Code Interpreter).



## Basic setup

Install packages

In [ ]:
!pip -qqq install japanize-matplotlib

Optional

In [ ]:
!pip -qqq install qrcode

Import libraries

In [ ]:
import os
from google.colab import userdata
import random
import time
from IPython.core.magic import register_cell_magic
import matplotlib.pyplot as plt
import japanize_matplotlib

import io
from contextlib import redirect_stdout

## Select AI

Select AI type and execute AI setup code which you select.

In [ ]:
AI_TYPE = 'LLAMA3' #@param ["ANTHROPIC", "OPENAI", "PHI3", "LLAMA3"] {allow-input: true}

### Anthropic setup

In [ ]:
!pip -qqq install anthropic

import anthropic

os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
client_anthropic = anthropic.Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))

def chat_claude(prompt_w_history, system_prompt):
    print("I am Claude")
    response = client_anthropic.messages.create(
        max_tokens=2048,
        messages=prompt_w_history,
        model="claude-3-opus-20240229",
        system=system_prompt
    )

    return response

### OpenAI setup

In [ ]:
!pip -qqq install openai

from openai import OpenAI

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
client_openai = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def chat_gpt(prompt_w_history):
    print("I am GPT")
    response = client_openai.chat.completions.create(
        model="gpt-4-turbo-2024-04-09",
        messages=prompt_w_history,
    )

    return response

### Phi-3 setup

In [ ]:
!pip install -qqq transformers accelerate

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct"
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)


def chat_phi3(prompt_w_history):
    print("I am Phi-3")

    prompt = tokenizer.apply_chat_template(prompt_w_history, tokenize=False, add_generation_prompt=True)

    # 推論の実行
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            do_sample=True,
            temperature=0.6,
            max_new_tokens=256,
        )
    response = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)

    return response

### Llama3 setup

In [ ]:
!pip install -qqq transformers accelerate bitsandbytes

import torch
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoModelForCausalLM, AutoTokenizer

login(token = userdata.get('HF_TOKEN'))

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct"
)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    torch_dtype="auto",
)


def chat_llama3(prompt_w_history):
    print("I am Llama 3")

    prompt = tokenizer.apply_chat_template(prompt_w_history, tokenize=False, add_generation_prompt=True)

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            max_new_tokens=256,
            eos_token_id=[
                tokenizer.eos_token_id,
                tokenizer.convert_tokens_to_ids("<|eot_id|>")
            ],
        )
    response = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)

    return response

## Code cook
Define function

In [ ]:
MAX_RETRAY_COUNT = 3

@register_cell_magic
def _(line, cell):
    system_prompt = f"""あなたはJupyterNotebookを使う優秀なPythonデータサイエンティストです。
    入力された指示をもとに、以下の要件に従ってデータ分析を実施してください。

    ###
    - 回答には、1つのPythonコードブロック（マジックコマンド、クラス定義、コンストラクタを含む）のみを含め、最初を```pythonで最後を```で囲ってください。
    - あなたは新しいPythonライブラリを自分で新たにインストールすることはできないので、ライブラリの不足時は、ユーザーにインストールする方法を提示してください。
    - 中身の分からないデータを操作するときは、まずはデータを確認するために読み込んだデータの情報(info, head)を出力するところまでだけを回答して。もう一度実行するようにユーザーにうながして下さい。
    - プロンプトにデータの情報が含まれていたら、その情報をもとに処理をすすめてください。
    """

    user_prompt = cell
    filename = line

    if filename != "":
        system_prompt += f"""
        # 入力データ:ファイルパス
        {filename}
        なお、データはUTF-8で読み込んでください。
        """

    if AI_TYPE == "OPENAI":
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    if AI_TYPE == "ANTHROPIC":
        messages = [
            {"role": "user", "content": user_prompt},
        ]
    if AI_TYPE == "PHI3":
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
         ]
    if AI_TYPE == "LLAMA3":
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
         ]

    prompt_w_history.extend(messages)

    retry_count = 0
    while retry_count < MAX_RETRAY_COUNT:
        try:
            if AI_TYPE == "OPENAI":
                response = chat_gpt(prompt_w_history)
            if AI_TYPE == "ANTHROPIC":
                response = chat_claude(prompt_w_history, system_prompt)
            if AI_TYPE == "PHI3":
                response = chat_phi3(prompt_w_history)
            if AI_TYPE == "LLAMA3":
                response = chat_llama3(prompt_w_history)

        except Exception as e:
            print(e)
            time.sleep(3)
            continue

        if AI_TYPE == "OPENAI":
            assistant_response = response.choices[0].message.content
        if AI_TYPE == "ANTHROPIC":
            assistant_response = response.content[0].text
        if AI_TYPE == "PHI3":
            assistant_response = response
        if AI_TYPE == "LLAMA3":
            assistant_response = response

        print("APIからの応答:", assistant_response)

        python = ""
        if "```python" in assistant_response and "```" in assistant_response:
            python = assistant_response.split("```python")[1].split("```")[0]

        # print("抽出されたPythonコード:", python)

        try:
            stream = io.StringIO()

            with redirect_stdout(stream):
                exec(python)

            output_stream = stream.getvalue()

            print("Result:")
            print(output_stream)

            prompt_w_history.append({"role": "assistant", "content": str(assistant_response) + 'result:' + output_stream})
            break
        except Exception as e:
            print(f"エラーが発生しました: {e}")
            prompt_w_history.append({"role": "assistant", "content": str(assistant_response)})
            retry_count += 1
            if retry_count < MAX_RETRAY_COUNT:
                prompt_w_history.append({"role": "user", "content": f"エラーが発生しました。エラーメッセージ: {e}\nコードを修正して再度実行してください。"})

    print("コードの実行が完了しました。" if retry_count < MAX_RETRAY_COUNT else "コードの修正に失敗しました。再実行回数が上限に達しました。")

Initialize prompt

In [ ]:
prompt_w_history = []

## Example

In [ ]:
%%_ sample_excel.xlsx
アップロードしたExcelファイルの金額を月ごとに集計して棒グラフにしてください。

In [ ]:
%%_
架空の国の地域ごとの人口比率のデータを可視化してください。日本語でお願いします。

## Reference:
- [claude_manim.ipynb](https://colab.research.google.com/github/wmoto-ai/claude-manim-notebook/blob/main/claude_manim.ipynb#scrollTo=mF8X8Q3LyZc9)
- [Google Colab で Llama 3 を試す](https://note.com/npaka/n/n73b0786f48e9)
- [Google Colab で Phi-3 を試す](https://note.com/npaka/n/n9915dbd44a84)